## Preambule

In [58]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Parameter

In [59]:
standard_settings = dict(Risk_of_exceedance='50%',
                         Negative_emissions='Medium',
                         Non_CO2_mitigation_potential='Medium',
                         Convergence_year=2050)
params = dict(start_year_analysis=2021)
rules = ['GF', 'PC', 'PCC', 'ECPC', 'AP', 'GDR']
rulecolors = ['steelblue', 'goldenrod', 'forestgreen', 'sienna', 'mediumvioletred']
costoptimal_colors = ['orangered', 'red', 'darkred']

## Paths

In [60]:
path_main = Path("X:/user/dekkerm/Projects/ECEMF_T5.2/")
path_data = path_main / "Data"
path_figs = path_main / "Figures" / "ECEMF_paper"

## Read data files

In [61]:
xr_total = xr.open_dataset(path_data / "xr_total.nc")
all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)

## Plot

Annotations and helper stuff

In [126]:
annotations = [
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=1.00, x=0.10, text="<b>(a) Grandfathering</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=1.00, x=0.74, text="<b>(b) Per Capita</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=0.655, x=0.10, text="<b>(c) Per Capita Convergence</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=0.655, x=0.91, text="<b>(d) Equal Cumulative per Capita</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=0.305, x=0.10, text="<b>(e) Ability to Pay</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=0.305, x=0.95, text="<b>(f) Greenhouse Development Rights</b>"),
    # dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=0.29, x=0.07, text="<b>(g) NDC reduction levels in 2030</b>"),
    # dict(xref='paper', yref='paper', ax=0, align='center', ay=-0, showarrow=False, textangle=-90, font=dict(color='black', size=12), y=0.175, x=0.02, text=">>> More reduction"),
    # dict(xref='paper', yref='paper', ax=0, align='center', ay=-0, showarrow=False, textangle=-90, font=dict(color='black', size=12), y=0.02, x=0.02, text="Less reduction <<<")
]

In [63]:
emis_red = (xr_total.CO2_hist.sel(Time=params['start_year_analysis']) - xr_total.CO2_ndc.sel(Time=2030)) / xr_total.CO2_ndc.sel(Time=2030)
emis_red_av = emis_red.mean(dim=['Conditionality', 'Hot_air', 'Ambition']).sel(Region=all_countries_iso)

In [64]:
cty_sort = np.array(emis_red_av.Region)[np.argsort(emis_red_av)]
wh = np.zeros(len(cty_sort))
for cty_i, cty in enumerate(cty_sort):
    if cty in ['USA', 'CHN', 'IND', 'DEU', 'FRA', 'AUS', 'CAN', 'RUS', 'BRA', 'ZAF', 'ARG', 'EGY', 'SAU', 'MEX', 'IDN', 'KAZ', "CHL", "KEN", 'TZA']:
        wh[cty_i] = 1
cty_sort_omit = np.copy(all_countries_names[np.argsort(emis_red_av)])
cty_sort_omit[wh ==0] = ''
cty_sort_omit = np.array([i.replace(' ', '<br>') for i in cty_sort_omit])
#cty_sort_omit[cty_sort_omit == "United States of America"] = 'United States<br>of America'
cols = np.array(['forestgreen']*len(cty_sort))
for cty_i, cty in enumerate(cty_sort):
    try:
        if cont_reg[cont_iso == cty] == 'Western Europe' or cont_reg[cont_iso == cty] == 'Eastern Europe' or cont_reg[cont_iso == cty] == 'Northern Europe' or cont_reg[cont_iso == cty] == 'Southern Europe' or cont_reg[cont_iso == cty] == 'Northern America':
            wh[cty_i] = 1
        elif cont_reg[cont_iso == cty] == 'Southern Asia' or cont_reg[cont_iso == cty] == 'Eastern Asia' or cont_reg[cont_iso == cty] == "Central Asia" or cont_reg[cont_iso == cty] == 'South-eastern Asia' or cont_reg[cont_iso == cty] == 'Western Asia':
            wh[cty_i] = 2
        elif cont_reg[cont_iso == cty] == 'Northern Africa' or cont_reg[cont_iso == cty] == "Sub-Saharan Africa":
            wh[cty_i] = 3
        elif cont_reg[cont_iso == cty] == 'Latin America and the Caribbean':
            wh[cty_i] = 4
    except:
        wh[cty_i] = -1
cols[wh == -1] = 'silver'
cols[wh == 0] = 'silver'
cols[wh == 1] = 'steelblue'
cols[wh == 2] = 'goldenrod'
cols[wh == 3] = 'tomato'
cols[wh == 4] = 'forestgreen'
textpos = np.array(['bottom center']*len(cty_sort))
textpos[wh != 3] = 'top center'
textpos[wh == 4] = 'bottom center'

In [127]:
fig = make_subplots(rows=3, cols=2, specs=[[{'type': 'choropleth'}, {'type': 'choropleth'}],
                                           [{'type': 'choropleth'}, {'type': 'choropleth'}],
                                           [{'type': 'choropleth'}, {'type': 'choropleth'}]],
                    horizontal_spacing = 0, vertical_spacing=0.0)

for rule_i, rule in enumerate(rules):
    if rule != 'ECPC': t = np.array(xr_total.sel(**standard_settings, Region=all_countries_iso).mean(dim=['Conditionality', 'Hot_air', 'Ambition', 'Scenario'])[rule+'_temp'])
    else: t=np.array([np.nan]*len(all_countries_iso))
    fig.add_trace(
        go.Choropleth(
        locations=all_countries_iso,
        z = t,
        locationmode = 'ISO-3',
        colorscale = 'RdYlGn_r',
        zmax = 3,
        zmin = 1.5,
        text = [str(r)+": "+str(np.round(float(t[r_i]), 2))+" C" for r_i, r in enumerate(all_countries_iso)],
        hovertemplate  = '%{text}',
        name="",
        marker_line_color='white', 
        marker_line_width=0.2,
        colorbar=dict(len=1, x=1, tickvals = np.arange(1.5, 4, 0.25), ticktext=[str(i)+'&deg; C' for i in np.arange(1.5, 4, 0.25)],
                      tickfont={'size':17}, title={"text":"<b>Temperature<br>rise<br>associated<br>with NDC", 'font': {'size':17}}),
        showscale=[True, False, False, False, False, False][rule_i]
    ), [1, 1, 2, 2, 3, 3][rule_i], [1, 2, 1, 2, 1, 2][rule_i])

fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                resolution=110,
                showcoastlines=False,)

fig.update_geos(visible=True, showcountries=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.update_layout(
    legend=dict(
        yanchor="top",
        y=0.29,
        xanchor="left",
        x=0.08,
        font=dict(size=15, color="black"),
    ),
    template='ggplot2',
    height=1000,
    width=1200,
    xaxis=go.XAxis(title='Country', showticklabels=False),
    yaxis=go.YAxis(
        title='GHG emission reduction in 2030 w.r.t. 2019', 
        range=(-1, 1),
        tickmode='array',
        tickvals=[-1, -0.75, -0.50, -0.25, 0, 0.25, 0.5, 0.75, 1],
        ticktext=['+100%', '+75%', '+50%', '+25%', '0%', '-25%', '-50%', '-75%', '100%']
    )
)
fig.update_xaxes(visible=False)

for ann in annotations:
    fig.add_annotation(**ann)
fig.show()

x:\user\dekkerm\environments\MyEnv\lib\site-packages\plotly\graph_objs\_deprecations.py:544: DeprecationWarning:

plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis


x:\user\dekkerm\environments\MyEnv\lib\site-packages\plotly\graph_objs\_deprecations.py:572: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis




In [128]:
fig.write_image(path_figs / "Figure_2.png", scale=3)